In [1]:
from ase.build import molecule
from ase.optimize import LBFGS
from ase import units

In [2]:
from qmlearn.api.api4ase import QMLCalculator
from qmlearn.drivers.mol import QMMol
from qmlearn.model import QMModel
from qmlearn.io import read_images
from qmlearn.preprocessing import AtomsCreater, build_train_atoms, build_properties

In [3]:
# basis = 'cc-pvtz'
basis = '6-31g*'
xc = 'b3lyp'
method = 'rks'

In [4]:
atoms = molecule('H2O')

In [5]:
refqmmol = QMMol(atoms = atoms, method = method, basis=basis, xc = xc)
atoms.calc = QMLCalculator(qmmodel = refqmmol, method = 'engine')

In [6]:
LBFGS(atoms).run(fmax=1E-5);

       Step     Time          Energy         fmax
LBFGS:    0 14:44:49    -2078.130668        0.0347
LBFGS:    1 14:44:50    -2078.130681        0.0119
LBFGS:    2 14:44:51    -2078.130684        0.0081
LBFGS:    3 14:44:52    -2078.130688        0.0000
LBFGS:    4 14:44:54    -2078.130688        0.0000


In [7]:
refqmmol = QMMol(atoms = atoms, method = method, basis=basis, xc = xc)
atoms.calc = QMLCalculator(qmmodel = refqmmol, method = 'engine', properties=('dipole',))

In [8]:
from ase.vibrations import Infrared

ir = Infrared(atoms, nfree=4)
ir.run()
ir.summary()

-------------------------------------
 Mode    Frequency        Intensity
  #    meV     cm^-1   (D/Å)^2 amu^-1
-------------------------------------
  0    1.8i     14.4i     6.8095
  1    1.7i     13.7i     0.0000
  2    0.0i      0.0i     0.0000
  3    0.0       0.0      0.0000
  4    0.0       0.0      0.0000
  5    1.6      12.9      2.4020
  6  212.0    1710.2      1.8790
  7  460.9    3717.6      0.0403
  8  476.3    3841.4      0.4696
-------------------------------------
Zero-point energy: 0.575 eV
Static dipole moment: 2.086 D
Maximum force on atom in `equilibrium`: 0.0000 eV/Å



In [9]:
vib = ir.get_vibrations()
modes = vib.get_modes()
frequencies = vib.get_frequencies().real

In [10]:
random_seed = 8888
temperature = 300
nsamples = 27
tol = 0.04

In [11]:
creater = AtomsCreater(modes=modes, frequencies=frequencies, 
                       atoms=atoms, temperature=temperature,
                       random_seed=random_seed)

In [12]:
images = build_train_atoms(creater, nsamples=nsamples, tol=tol, refatoms=atoms)

Start build
Get 27 samples at 40258 step.


In [13]:
prop = ['vext', 'gamma', 'energy', 'forces', 'dipole', 'ke']
properties = build_properties(images, refqmmol = refqmmol, properties=prop)

  0%|          | 0/27 [00:00<?, ?it/s]

In [14]:
from qmlearn.io import write_db
write_db('h2o_300_0.040_27_b3lyp_qmldb.hdf5', refqmmol, images, properties)

['rks', 'rks/qmmol', 'rks/train_atoms_27', 'rks/train_props_27']


In [15]:
atoms.write('opt-b3lyp.xyz', format='xyz')